In [1]:
!pip install -e .

Obtaining file:///mnt/ssd-1/soar-automated_interpretability/graphs/christopher/Delphi_Cluster
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for co-activation-grouping (pyproject.toml) ... done
  Created wheel for co-activation-grouping: filename=co_activation_grouping-0.1.0-0.editable-py3-none-any.whl size=1668 sha256=cc0468bcc80a6ace7f6d9d2ef18fcdcd7f5aaa3e404fd984be4133cc4f9eace5
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ab2eql2/wheels/af/73/e5/e823dc85c1a0456beca7e8a6e8def4b2e8f37350d7136af80f
Successfully built co-activation-grouping
  Attempting uninstall: co-activation-grouping
    Found existing installation: co-activation-grouping 0.1.0
    Uninstalling co-activation-grouping-0.1.0:
      Successfully uninstalled co-activation-grouping-0.1.0


In [2]:
from delphi.config import ConstructorConfig, SamplerConfig
from delphi.latents import (
    ActivatingExample,
    LatentDataset,
    LatentRecord,
)
import argparse
import torch
import os
import json
from collections import defaultdict
from safetensors import safe_open
import math
from huggingface_hub import login
from tqdm import tqdm

/mnt/ssd-1/soar-automated_interpretability/graphs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
token = # token

In [ ]:
login(token=token)


In [ ]:
# Load attribution graph and extract features by layer

# GRAPH_PATH = # for attribution
# CACHE_PATH = # for cache

# Load the attribution graph
with open(GRAPH_PATH, "r") as f:
    graph_data = json.load(f)

print(f"Prompt: {graph_data['metadata']['prompt']}")
print(f"Total nodes: {len(graph_data['nodes'])}")

Prompt: <bos>A bee is a type of
Total nodes: 443


In [5]:
graph_data["nodes"][20]

{'feature_type': 'cross layer transcoder',
 'layer': '0',
 'node_id': 'intermediate_0_0_12640',
 'jsNodeId': 'intermediate_0_0_12640-0',
 'feature': 79903760,
 'ctx_idx': 1,
 'run_idx': 0,
 'reverse_ctx_idx': 0,
 'clerp': '',
 'token_prob': 0.0,
 'is_target_logit': False,
 'influence': 0.6330530617407577}

In [6]:
def get_attribution_features(GRAPH_PATH: str):
  # path to the attribution graph json
  with open(GRAPH_PATH, "r") as f:
    graph_data = json.load(f)
  # gets all the features in the attribution graph and filters out error nodes features
  attribution_features = set([data["feature"] for data in graph_data["nodes"]])
  attribution_features.remove(None)
  return attribution_features


In [7]:
attribution_features = get_attribution_features(GRAPH_PATH)

In [8]:
def cantor(num1, num2):
    return (num1 + num2) * (num1 + num2 + 1) // 2 + num2


def cantor_decode(num):
    w = math.floor((math.sqrt(8 * num + 1) - 1) / 2)
    t = (w * w + w) // 2
    y = num - t
    x = w - y
    return x, y

In [9]:
directory_path = "/mnt/ssd-1/soar-automated_interpretability/graphs/pawan/results/gemmma-scope-2b-pt-transcoders" # Replace with your desired directory
entries = os.listdir(directory_path)


latent_dict = {
  layer: torch.arange(0, 16384) for layer in entries
}

sampler_cfg = SamplerConfig()
constructor_cfg = ConstructorConfig()

# Delphi_Cluster_Co_Activations/src/data/llama-3-8B change this to raw dir?
dataset = LatentDataset(
    raw_dir= "/mnt/ssd-1/soar-automated_interpretability/graphs/pawan/delphi/results/gemma2b_transcoder-sparsify-1m_cache",
    modules=entries, # This a list of the different caches to load from
    sampler_cfg=sampler_cfg,
    constructor_cfg=constructor_cfg,
    latents=latent_dict,
    tokenizer=None
)


In [10]:
def collect_attribution_graph_delphi(dataset: LatentDataset, attribution_features: set) -> dict:
  node_data = defaultdict(dict)
  for b in dataset.buffers:
    # Load raw data for all latents in this buffer
    latents, split_locations, split_activations = b.load_data_per_latent()
    current_layer = int(b.module_path.split(".")[1]) # might change depending on model
    for i, latent in enumerate(latents):

      # cantor(layer, feature) in delphi it was
      delphi_feature = int(cantor(current_layer, latent))
      #if feature in attribution graph store it
      if delphi_feature in attribution_features:
          latent_locations = split_locations[i]
          latent_activations = split_activations[i]
          node_data[delphi_feature] = {
            "layer": current_layer,
            "layer_latent/feature": latent,
            "activations": latent_activations,
            "locations": latent_locations,
          }
  return node_data


In [ ]:
# the features in latent dict in the attribution graph
nodedata = collect_attribution_graph_delphi(dataset, attribution_features)

In [12]:
cantor_ids = list(nodedata.keys())
cantor_to_idx = {cid: idx for idx, cid in enumerate(cantor_ids)}
n_features = len(cantor_ids)
all_feature_indices = []
all_position_indices = []

In [ ]:
# makes a dataset of feature, locations
for cantor_id, data in tqdm(nodedata.items(), desc="Processing features"):
    feature_idx = cantor_to_idx[cantor_id]
    locations = data["locations"]  # Shape: [N, 2] or [N, 3] -> (batch, token_pos, ...)
    
    # Convert locations to tensor if needed
    if not isinstance(locations, torch.Tensor):
        locations = torch.tensor(locations)
    
    # Extract batch and token position
    batch_indices = locations[:, 0].long()
    token_indices = locations[:, 1].long()
    
    # Create unique position ID using Cantor pairing
    position_cantor = ((batch_indices + token_indices) * (batch_indices + token_indices + 1)) // 2 + token_indices
    # Store pairs
    all_feature_indices.extend([feature_idx] * len(position_cantor))
    all_position_indices.extend(position_cantor.tolist())

Processing features: 100%|██████████| 299/299 [00:00<00:00, 401.70it/s]


In [ ]:
# one tensor with all features one with positions * features
feature_indices = torch.tensor(all_feature_indices, dtype=torch.long)
position_indices = torch.tensor(all_position_indices, dtype=torch.long)

In [ ]:
unique_positions, position_indices = torch.unique(position_indices, return_inverse=True)
# matrix latent, position_cantor (batch, position)    
sparse_matrix = torch.sparse_coo_tensor(
    torch.stack([feature_indices, position_indices]),
    torch.ones(len(feature_indices)),
    (n_features, len(unique_positions))
)

co_occurrence_matrix = (sparse_matrix @ sparse_matrix.T).to_dense()

In [16]:
co_occurrence_matrix

tensor([[9.9580e+03, 1.4840e+03, 5.9000e+01,  ..., 1.1000e+01, 1.3700e+02,
         1.1410e+03],
        [1.4840e+03, 1.0510e+04, 2.6000e+01,  ..., 3.1000e+01, 1.3000e+01,
         1.3760e+03],
        [5.9000e+01, 2.6000e+01, 3.8140e+03,  ..., 1.0000e+00, 4.0200e+02,
         1.0000e+00],
        ...,
        [1.1000e+01, 3.1000e+01, 1.0000e+00,  ..., 9.3820e+03, 4.0000e+00,
         1.4000e+01],
        [1.3700e+02, 1.3000e+01, 4.0200e+02,  ..., 4.0000e+00, 9.4070e+03,
         8.0000e+00],
        [1.1410e+03, 1.3760e+03, 1.0000e+00,  ..., 1.4000e+01, 8.0000e+00,
         3.4930e+03]])

In [ ]:
# from delphi
def compute_jaccard(cooc_matrix):
        self_occurrence = cooc_matrix.diagonal()
        jaccard_matrix = cooc_matrix / (
            self_occurrence[:, None] + self_occurrence - cooc_matrix
        )
        # remove the diagonal and keep the upper triangle
        return jaccard_matrix

In [34]:
# dim is feature by feature
jaccard = compute_jaccard(co_occurrence_matrix)

In [29]:
number_of_neighbours = 10

In [36]:
top_k_indices, values = torch.topk(
          jaccard, number_of_neighbours + 1, dim=1
      )
# top k for each feature

In [ ]:
neighbours_list = {
            i: list(zip(top_k_indices[i].tolist()[1:], values[i].tolist()[1:]))
            for i in range(len(top_k_indices))
        }
# stores 10 neighboors and the amount for each 

In [38]:
neighbours_list

{0: [(0.11041465401649475, 231),
  (0.09606769680976868, 204),
  (0.09268887341022491, 298),
  (0.07817108929157257, 1),
  (0.07045049965381622, 33),
  (0.06387481838464737, 244),
  (0.05688665807247162, 13),
  (0.054848574101924896, 98),
  (0.05268091335892677, 47),
  (0.05178816616535187, 294)],
 1: [(0.12515327334403992, 204),
  (0.12265624850988388, 231),
  (0.11575562506914139, 250),
  (0.11566410958766937, 35),
  (0.11046997457742691, 244),
  (0.10897283256053925, 298),
  (0.09801813215017319, 270),
  (0.09508318454027176, 47),
  (0.09336148202419281, 284),
  (0.09316717833280563, 191)],
 2: [(0.1886715590953827, 42),
  (0.14593824744224548, 195),
  (0.13617326319217682, 21),
  (0.13295112550258636, 247),
  (0.12361296266317368, 161),
  (0.11256467550992966, 96),
  (0.11209355294704437, 238),
  (0.10929405689239502, 123),
  (0.10851319134235382, 293),
  (0.09872611612081528, 214)],
 3: [(0.060152728110551834, 291),
  (0.052170537412166595, 92),
  (0.04691128805279732, 169),
  (0.

In [17]:
# used to find 16384
directory_path = "/mnt/ssd-1/soar-automated_interpretability/graphs/pawan/results/gemmma-scope-2b-pt-transcoders"
test_layer = "layers.6.mlp"
layer_path = os.path.join(directory_path, test_layer)

file_path = os.path.join(layer_path, "sae.safetensors")

with safe_open(file_path, framework="pt") as f:
    print("Tensor shapes:")
    for key in f.keys():
        tensor = f.get_tensor(key)
        print(f"  {key}: {tensor.shape}")
    
    # The number of features is typically the first dimension of W_dec
    # or the output dimension of encoder.weight
    W_dec = f.get_tensor("W_dec")
    print(f"\n✅ Number of features: {W_dec.shape[0]}")

Tensor shapes:
  W_dec: torch.Size([16384, 2304])
  b_dec: torch.Size([2304])
  encoder.bias: torch.Size([16384])
  encoder.weight: torch.Size([16384, 2304])
  post_enc: torch.Size([16384])

✅ Number of features: 16384
